In [1]:
import numpy as np
import itertools

import cirq
from cirq import ops, Circuit
from cirq.ops import CZ, H, CNOT, X, Y, Z, SWAP

from cirq.contrib.rearrange.separate import convert_circuit, non_clifford_half, clifford_half

from cirq.contrib.rearrange.axis import *
from cirq.contrib.rearrange.clifford_pauli_gate import CliffordPauliGate
from cirq.contrib.rearrange.interaction_gate import InteractionGate
from cirq.contrib.rearrange.pauli_string import PauliString
from cirq.contrib.rearrange.non_clifford_gate import NonCliffordGate

from cirq.testing import assert_allclose_up_to_global_phase

In [2]:
q0, q1, q2 = (cirq.NamedQubit('q{}'.format(i)) for i in range(3))

In [3]:
def qft_ops(qubits, use_partial_cz=True, use_h=True):
    if use_partial_cz:
        def cz_gate(q0, q1, half_turns):
            yield CZ(q0, q1) ** half_turns
    else:
        cz_gate = partial_cz_decompose
    if use_h:
        def h_gate(q0):
            yield H(q0)
    else:
        def h_gate(q0):
            yield Y(q0) ** 0.5
            yield X(q0)
    n = len(qubits)
    if n <= 0:
        return
    for i in range(n):
        for j in range(0, i):
            yield from cz_gate(qubits[i], qubits[j], 1/2**(i-j))
        yield from h_gate(qubits[i])
        
def qft_circuit(n, **kwargs):
    qubits = tuple((cirq.NamedQubit('q{}'.format(i)) for i in range(n)))
    return Circuit.from_ops(qft_ops(qubits, **kwargs))
        
def partial_cz_decompose(q0, q1, half_turns):
    yield X(q0) ** 0.5
    yield X(q1)
    yield CZ(q0, q1)
    yield Y(q0) ** (-half_turns/2)
    yield CZ(q0, q1)
    yield X(q0) ** -0.5
    yield X(q1)
    yield Z(q0) ** (half_turns/2)
    yield Z(q1) ** (half_turns/2)

circuit_orig = qft_circuit(4, use_partial_cz=False, use_h=False)
circuit_orig

q0: ───Y^0.5───X───────X───Z─────────────Z───X────────Z^0.25───────────X───────Z──────────────Z───X────────Z^0.125───────────────────────────────────────────────────────X───────Z───────────────Z───X────────Z^0.0625─────────────────────────────────────────────────────────────────────────────────────────────────────────
                           │             │                                     │              │                                                                                  │               │
q1: ───────────X^0.5───────@───Y^-0.25───@───X^-0.5───Z^0.25───Y^0.5───X───────┼──────────────┼──────────────────────X───────Z─────────────Z───X────────Z^0.25───────────────────┼───────────────┼───────────────────────X───────Z──────────────Z───X────────Z^0.125───────────────────────────────────────────────────────────
                                                                               │              │                              │             │                                     │               │                               │              │
q2: ───────────────────────────────────────────────────────────────────X^0.5───@───Y^-0.125───@───X^-0.5───Z^0.125───X^0.5───@───Y^-0.25───@───X^-0.5───Z^0.25───Y^0.5───X───────┼───────────────┼───────────────────────────────┼──────────────┼──────────────────────X───────Z─────────────Z───X────────Z^0.25───────────────
                                                                                                                                                                                 │               │                               │              │                              │             │
q3: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X^0.5───@───Y^-0.0625───@───X^-0.5───Z^0.0625───X^0.5───@───Y^-0.125───@───X^-0.5───Z^0.125───X^0.5───@───Y^-0.25───@───X^-0.5───Z^0.25───Y^0.5───X───

In [4]:
circuit = convert_circuit(circuit_orig)
circuit

q0: ───Y^0.5───X───────X───Z───────────────Z───X────────[Z]^0.25───────────X───────Z────────────────Z───X────────[Z]^0.125───────────────────────────────────────────────────────────X───────Z─────────────────Z───X────────[Z]^0.0625─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                           │               │                                       │                │                                                                                        │                 │
q1: ───────────X^0.5───────Z───[Y]^-0.25───Z───nX^0.5───[Z]^0.25───Y^0.5───X───────┼────────────────┼────────────────────────X───────Z───────────────Z───X────────[Z]^0.25───────────────────┼─────────────────┼─────────────────────────X───────Z────────────────Z───X────────[Z]^0.125───────────────────────────────────────────────────────────────
                                                                                   │                │                                │               │                                       │                 │                                 │                │
q2: ───────────────────────────────────────────────────────────────────────X^0.5───Z───[Y]^-0.125───Z───nX^0.5───[Z]^0.125───X^0.5───Z───[Y]^-0.25───Z───nX^0.5───[Z]^0.25───Y^0.5───X───────┼─────────────────┼─────────────────────────────────┼────────────────┼────────────────────────X───────Z───────────────Z───X────────[Z]^0.25───────────────
                                                                                                                                                                                             │                 │                                 │                │                                │               │
q3: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X^0.5───Z───[Y]^-0.0625───Z───nX^0.5───[Z]^0.0625───X^0.5───Z───[Y]^-0.125───Z───nX^0.5───[Z]^0.125───X^0.5───Z───[Y]^-0.25───Z───nX^0.5───[Z]^0.25───Y^0.5───X───

In [5]:
c_left = non_clifford_half(circuit)
c_left

q0: ───[nX]^-0.25──────────────[X]^0.25───[nX]^-0.125───────────────[X]^0.125──────────────────────────────────────[nX]^-0.0625────────────────[X]^0.0625────────────────────────────────────────────────────────────────────────────
       │                                  │                                                                        │
q1: ───[nZ]─────────[Z]^0.25──────────────┼─────────────────────────────────────[nX]^-0.25──────────────[X]^0.25───┼────────────────────────────────────────[nX]^-0.125───────────────[X]^0.125──────────────────────────────────────
                                          │                                     │                                  │                                        │
q2: ──────────────────────────────────────[nZ]──────────[Z]^0.125───────────────[nZ]─────────[Z]^0.25──────────────┼────────────────────────────────────────┼─────────────────────────────────────[nX]^-0.25──────────────[X]^0.25───
                                                                                                                   │                                        │                                     │
q3: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────[nZ]───────────[Z]^0.0625────────────────[nZ]──────────[Z]^0.125───────────────[nZ]─────────[Z]^0.25──────────────

In [6]:
c_right = clifford_half(circuit)
c_right

q0: ───Y^0.5───X───────X───Z───Z───X────────────────X───────Z───Z───X─────────────────────────────────────────X───────Z───Z───X──────────────────────────────────────────────────────────────────────
                           │   │                            │   │                                                     │   │
q1: ───────────X^0.5───────Z───Z───nX^0.5───Y^0.5───X───────┼───┼────────────X───────Z───Z───X────────────────────────┼───┼────────────X───────Z───Z───X─────────────────────────────────────────────
                                                            │   │                    │   │                            │   │                    │   │
q2: ────────────────────────────────────────────────X^0.5───Z───Z───nX^0.5───X^0.5───Z───Z───nX^0.5───Y^0.5───X───────┼───┼────────────────────┼───┼────────────X───────Z───Z───X────────────────────
                                                                                                                      │   │                    │   │                    │   │
q3: ──────────────────────────────────────────────────────────────────────────────────────────────────────────X^0.5───Z───Z───nX^0.5───X^0.5───Z───Z───nX^0.5───X^0.5───Z───Z───nX^0.5───Y^0.5───X───

In [13]:
assert_allclose_up_to_global_phase(circuit_orig.to_unitary_matrix(),
                                   (c_left + c_right).to_unitary_matrix(),
                                   atol=1e-7, rtol=1e-7)

In [7]:
(c_left + c_right).to_unitary_matrix().round(5)

array([[-0.25   -0.j     , -0.25   +0.j     , -0.25   +0.j     ,
        -0.25   -0.j     , -0.25   -0.j     , -0.25   -0.j     ,
        -0.25   -0.j     , -0.25   -0.j     , -0.25   -0.j     ,
        -0.25   +0.j     , -0.25   +0.j     , -0.25   -0.j     ,
        -0.25   -0.j     , -0.25   -0.j     , -0.25   -0.j     ,
        -0.25   -0.j     ],
       [-0.25   -0.j     ,  0.25   -0.j     , -0.25   +0.j     ,
         0.25   +0.j     , -0.25   -0.j     ,  0.25   +0.j     ,
        -0.25   -0.j     ,  0.25   +0.j     , -0.25   -0.j     ,
         0.25   -0.j     , -0.25   +0.j     ,  0.25   +0.j     ,
        -0.25   -0.j     ,  0.25   +0.j     , -0.25   -0.j     ,
         0.25   +0.j     ],
       [-0.25   -0.j     , -0.     -0.25j   ,  0.25   +0.j     ,
         0.     +0.25j   , -0.25   -0.j     ,  0.     -0.25j   ,
         0.25   +0.j     , -0.     +0.25j   , -0.25   -0.j     ,
        -0.     -0.25j   ,  0.25   +0.j     ,  0.     +0.25j   ,
        -0.25   -0.j     ,  0.    

In [14]:
c_left + c_right

q0: ───[nX]^-0.25──────────────[X]^0.25───[nX]^-0.125───────────────[X]^0.125──────────────────────────────────────[nX]^-0.0625────────────────[X]^0.0625────────────────────────────────────────────────────────────────────────────Y^0.5───X───────X───Z───Z───X────────────────X───────Z───Z───X─────────────────────────────────────────X───────Z───Z───X──────────────────────────────────────────────────────────────────────
       │                                  │                                                                        │                                                                                                                                     │   │                            │   │                                                     │   │
q1: ───[nZ]─────────[Z]^0.25──────────────┼─────────────────────────────────────[nX]^-0.25──────────────[X]^0.25───┼────────────────────────────────────────[nX]^-0.125───────────────[X]^0.125──────────────────────────────────────────────X^0.5───────Z───Z───nX^0.5───Y^0.5───X───────┼───┼────────────X───────Z───Z───X────────────────────────┼───┼────────────X───────Z───Z───X─────────────────────────────────────────────
                                          │                                     │                                  │                                        │                                                                                                                             │   │                    │   │                            │   │                    │   │
q2: ──────────────────────────────────────[nZ]──────────[Z]^0.125───────────────[nZ]─────────[Z]^0.25──────────────┼────────────────────────────────────────┼─────────────────────────────────────[nX]^-0.25──────────────[X]^0.25────────────────────────────────────────────────X^0.5───Z───Z───nX^0.5───X^0.5───Z───Z───nX^0.5───Y^0.5───X───────┼───┼────────────────────┼───┼────────────X───────Z───Z───X────────────────────
                                                                                                                   │                                        │                                     │                                                                                                                                                 │   │                    │   │                    │   │
q3: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────[nZ]───────────[Z]^0.0625────────────────[nZ]──────────[Z]^0.125───────────────[nZ]─────────[Z]^0.25─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X^0.5───Z───Z───nX^0.5───X^0.5───Z───Z───nX^0.5───X^0.5───Z───Z───nX^0.5───Y^0.5───X───

In [15]:
qft_circuit(4)

q0: ───H───Z^0.5───────Z^0.25───────────────Z^0.125────────────────────────
           │           │                    │
q1: ───────@───────H───┼────────Z^0.5───────┼─────────Z^0.25───────────────
                       │        │           │         │
q2: ───────────────────@────────@───────H───┼─────────┼────────Z^0.5───────
                                            │         │        │
q3: ────────────────────────────────────────@─────────@────────@───────H───